# intermediate_SARC
This notebook takes our custom XED binary datas_et and trains an intermediate model.

## Imports & Settings

First, update working directory to parent so that we may use our custom functions

In [1]:
import os
os.chdir('..')
# os.getcwd( )

In [3]:
import params
from utils import *
from trainer import *

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

# suppress model warning
from transformers import logging
logging.set_verbosity_error()

# set logging level
import logging
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [ ]:
# set general seeds
set_seeds(1)

# set dataloader generator seed
g = torch.Generator()
g.manual_seed(1)

# set params for this model
params.num_labels = 2
params.output_dir = "model_saves/intermediate_SARC_01"

# Ensure we're on an ARM environment if necessary.
platform_check()

We're Armed: macOS-13.1-arm64-i386-64bit


## Load Data

### SARC

In [4]:
dataset_path = 'data/inter_SARC/SARC_preped_sampled_train.csv'
df = pd.read_csv(dataset_path)

df.head()

,label,text,author,subreddit,score,ups,downs,date,created_utc,parent_comment,num_word_text
0,0,That's paid propaganda created by Big Law Enfo...,akronix10,news,15,15,0,2014-11,2014-11-26 20:50:39,"If TV tells me anything, most beat cops hate i...",14
1,0,I imagine they'll use it for regenerative brak...,f03nix,Futurology,1,1,0,2015-07,2015-07-20 07:30:02,"Ummm, powered by a wind turbine? Sorry...but t...",8
2,0,Distraction and diversion from the real proble...,Shinranshonin,politics,1,-1,-1,2016-12,2016-12-12 13:51:13,"The media call Trump a 'cyberbully,' even when...",28
3,0,Not at this point in the movie,spot35,funny,2,2,0,2015-11,2015-11-21 07:59:49,"But it's a protocol droid, fluent in over 6 mi...",7
4,0,I have a feeling you don't understand the Mand...,throwaway-11-20-2016,MandelaEffect,1,-1,-1,2016-12,2016-12-28 16:14:36,so should the side bar say that a large number...,10


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   label           100000 non-null  int64 
 1   text            100000 non-null  object
 2   author          100000 non-null  object
 3   subreddit       100000 non-null  object
 4   score           100000 non-null  int64 
 5   ups             100000 non-null  int64 
 6   downs           100000 non-null  int64 
 7   date            100000 non-null  object
 8   created_utc     100000 non-null  object
 9   parent_comment  100000 non-null  object
 10  num_word_text   100000 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 8.4+ MB


In [6]:
df['label'].value_counts()

0    50000
1    50000
Name: label, dtype: int64

### Target Text & Labels

In [7]:
text = df.text.values
labels = df.label.values

## Preprocess

In [8]:
token_id = []
attention_masks = []

for sample in text:
  encoding_dict = preprocessing(sample, params.tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

## Data Split
We split the dataset into train (80%) and validation (20%) sets, and wrap them around a torch.utils.data.DataLoader object.

In [9]:
val_ratio = 0.2

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels,
    random_state=1)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = RandomSampler(val_set),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
        )

## Train

Download transformers.RobertaForSequenceClassificatio, which is a RoBERTa model with a linear layer for sentence classification (or regression) on top of the pooled output:

In [10]:
# Load the RobertaForSequenceClassification model
model = RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                         num_labels = params.num_labels,
                                                         output_attentions = False,
                                                         output_hidden_states = False,
                                                         )

from torchinfo import summary
summary(model, input_size=(1, 512), dtypes=['torch.IntTensor'])

Layer (type:depth-idx)                                       Output Shape              Param #
RobertaForSequenceClassification                             [1, 2]                    --
├─RobertaModel: 1-1                                          [1, 512, 768]             --
│    └─RobertaEmbeddings: 2-1                                [1, 512, 768]             --
│    │    └─Embedding: 3-1                                   [1, 512, 768]             38,603,520
│    │    └─Embedding: 3-2                                   [1, 512, 768]             768
│    │    └─Embedding: 3-3                                   [1, 512, 768]             394,752
│    │    └─LayerNorm: 3-4                                   [1, 512, 768]             1,536
│    │    └─Dropout: 3-5                                     [1, 512, 768]             --
│    └─RobertaEncoder: 2-2                                   [1, 512, 768]             --
│    │    └─ModuleList: 3-6                                  --               

Set model to device, initialize trainer

In [11]:
model.to(params.device)
print(f"Trained Dataset: {dataset_path}")
print(f"Device: {params.device}")

optimizer = torch.optim.Adam(params=model.parameters(), lr=params.learning_rate) #roberta

trainer = Trainer(model=model,
                  device=params.device,
                  tokenizer=params.tokenizer,
                  train_dataloader=train_dataloader,
                  validation_dataloader=validation_dataloader,
                  epochs=params.epochs,
                  optimizer=optimizer,
                  val_loss_fn=params.val_loss_fn,
                  num_labels=params.num_labels,
                  output_dir=params.output_dir,
                  save_freq=params.save_freq,
                  checkpoint_freq=params.checkpoint_freq)

Trained Dataset: data/SARC/SARC_preped_sampled_train.csv
Device: mps


Fit the model to our training data.

In [12]:
trainer.fit()

Epoch 1:   0%|          | 0/5000 [00:00<?, ?batch/s]/Users/jarradjinx/opt/anaconda3/envs/itesd_env/lib/python3.9/site-packages/transformers/models/roberta/modeling_roberta.py:1570: UserWarning: The operator 'aten::cumsum.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  incremental_indices = (torch.cumsum(mask, dim=1).type_as(mask) + past_key_values_length) * mask
	 Validation 1249: 100%|██████████| 1250/1250 [08:19<00:00,  2.50batch/s]


	 - Train loss: 0.552180
	 - Validation Loss: 0.507575
	 - Validation Accuracy: 0.746200
	 - Validation F1: 0.711028
	 - Validation Recall: 0.671129
	 - Validation Precision: 0.789116
	 * Model @ epoch 1 saved to model_saves/intermediate_SARC_01/E01_A0.75_F0.71


	 Validation 1249: 100%|██████████| 1250/1250 [08:20<00:00,  2.50batch/s]


	 - Train loss: 0.478418
	 - Validation Loss: 0.511777
	 - Validation Accuracy: 0.754750
	 - Validation F1: 0.725020
	 - Validation Recall: 0.696219
	 - Validation Precision: 0.788127
	 * Model @ epoch 2 saved to model_saves/intermediate_SARC_01/E02_A0.75_F0.73


	 Validation 1249: 100%|██████████| 1250/1250 [08:23<00:00,  2.48batch/s]


...

	 - Train loss: 0.154791
	 - Validation Loss: 0.861222
	 - Validation Accuracy: 0.739600
	 - Validation F1: 0.729675
	 - Validation Recall: 0.750797
	 - Validation Precision: 0.733539
	 * Model @ epoch 8 saved to model_saves/intermediate_SARC_01/E08_A0.74_F0.73


	 Validation 1249: 100%|██████████| 1250/1250 [08:21<00:00,  2.49batch/s]


	 - Train loss: 0.130389
	 - Validation Loss: 0.927679
	 - Validation Accuracy: 0.745100
	 - Validation F1: 0.737106
	 - Validation Recall: 0.762999
	 - Validation Precision: 0.737391
	 * Model @ epoch 9 saved to model_saves/intermediate_SARC_01/E09_A0.75_F0.74


	 Validation 1249: 100%|██████████| 1250/1250 [08:30<00:00,  2.45batch/s]


	 - Train loss: 0.109669
	 - Validation Loss: 1.007827
	 - Validation Accuracy: 0.741000
	 - Validation F1: 0.727307
	 - Validation Recall: 0.740828
	 - Validation Precision: 0.739979
	 * Model @ epoch 10 saved to model_saves/intermediate_SARC_01/E10_A0.74_F0.73
